In [1]:
import numpy as np
import os
from pathlib import Path
import sys
from tensorflow.keras.preprocessing.image import load_img, img_to_array

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import src.false_labels_effect.util as util
import src.false_labels_effect.data_loader as dl
import src.false_labels_effect.models as mdls

In [2]:
# TODO: Select model task
#   'Class': main class classification
#   'Subclass': sub class classification
#   'Annotations' : polygon vertices prediction
model_task = 'Class'

# TODO: set number of images
limit_loaded_images = 10  # use None for "all" images

In [3]:
# set training and test img png path
train_img_png_path = Path("..\\data\\Images_4c_Poly\\Train")
test_img_png_path = Path("..\\data\\Images_4c_Poly\\Test")

# set training and test img npy path
img_npy_path = Path('..\\data\\Images_4c_Poly\\Train_Test_npy')

# set label path
train_label_path = Path("..\\data\\Labels_4c_Poly\\Train.npy")
test_label_path = Path("..\\data\\Labels_4c_Poly\\Test.npy")

In [4]:

# load labels dict
train_labels_dict = util.load_labels(train_label_path)
test_labels_dict = util.load_labels(test_label_path)


In [5]:
# TODO: set size of images 
resize_to = (244, 244)

# load train images, resize and save as npy
if not os.path.exists(f'{img_npy_path}'):
    os.mkdir(img_npy_path)

    i = 0
    for image_path in train_img_png_path.iterdir():
        i += 1
        if limit_loaded_images is not None and i > limit_loaded_images:
            break

        # Load without resizing so that polygon fits (for now)
        img_id = image_path.name.split(".")[0]
        img = load_img(image_path)

        # Use util resize function resize image and polygon
        # TODO: poly resize currently not saved
        img_res, poly_res = util.resize(
            img, train_labels_dict[img_id], resize_to
        )

        npy_img = img_to_array(img_res)
        np.save(f'{img_npy_path}\\{img_id}', npy_img)

    i = 0

    for image_path in test_img_png_path.iterdir():
        i += 1
        if limit_loaded_images is not None and i > limit_loaded_images:
            break

        # Load without resizing so that polygon fits (for now)
        img_id = image_path.name.split(".")[0]
        img = load_img(image_path)

        # Use util resize function resize image and polygon
        # TODO: poly resize currently not saved
        img_res, poly_res = util.resize(
            img, test_labels_dict[img_id], resize_to
        )

        npy_img = img_to_array(img_res)
        np.save(f'{img_npy_path}\\{img_id}', npy_img)


In [6]:
# create dict of included train and test images formated for keras data loader
partition = {}
img_ids_included = [str(i.name).split(".")[0] for i in img_npy_path.iterdir()]
partition['train'] = [id for id in img_ids_included if 'Train' in id]
partition['test'] = [id for id in img_ids_included if 'Test' in id]

In [7]:
# filter train labels to only include transformed images
train_labels_dict_incl = {}
for (key, value) in train_labels_dict.items():
    if key in partition['train']:
        train_labels_dict_incl[key] = value

# filter test labels to only include transformed images
test_labels_dict_incl = {}
for (key, value) in test_labels_dict.items():
    if key in partition['test']:
        test_labels_dict_incl[key] = value

# generate flattened dict of model task corresponding labels
train_labels_dict_flat = util.select_label(train_labels_dict_incl, model_task)
test_labels_dict_flat = util.select_label(test_labels_dict_incl, model_task)

# encode categorical labels for classification tasks
if model_task in ['Class', 'Subclass']:
    train_labels_dict_flat, test_labels_dict_flat = util.encode_labels(train_labels_dict_flat, test_labels_dict_flat)

In [8]:
# define data loader parameters
batch_size = 10

params = {'dim': (resize_to[0],resize_to[0]),
          'batch_size': batch_size,
          'n_classes': 4,
          'n_channels': 3,
          'shuffle': True}

# load data
training_loader = dl.DataLoader(partition['train'], train_labels_dict_flat, **params)
test_loader = dl.DataLoader(partition['test'], test_labels_dict_flat, **params)

In [9]:
# load model parameter
multiprocessing = True
n_workers = 2

# load models
basic_cnn = mdls.create_cnn_model(resize_to)

# compile models and train
basic_cnn.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

basic_cnn.fit(x = training_loader,
            #   validation_data = test_loader,
              epochs = 5,
              use_multiprocessing = multiprocessing,
              workers = n_workers,
              verbose = 1)

Epoch 1/5
1/1 [==============================] - 0s 3ms/step - loss: 10.9407 - accuracy: 0.2000
Epoch 2/5
1/1 [==============================] - 0s 5ms/step - loss: 48689.6523 - accuracy: 0.3000
Epoch 3/5
1/1 [==============================] - 0s 9ms/step - loss: 10904.3330 - accuracy: 0.4000
Epoch 4/5
1/1 [==============================] - 0s 6ms/step - loss: 1120.6929 - accuracy: 0.1000
Epoch 5/5
1/1 [==============================] - 0s 5ms/step - loss: 1033.3845 - accuracy: 0.3000


In [12]:
# show test accuracy
score = basic_cnn.evaluate(x = test_loader,
                           batch_size = batch_size,
                           use_multiprocessing = multiprocessing,
                           workers = n_workers,
                           verbose = 0)
print('Test accuracy:', score[1])

Test accuracy: 0.6000000238418579
